<center><h1>Martinez_Ariel_Final_Project</h1></center>

Name: Ariel Martinez Birlanga
<br>
Github Username: ambirlanga
<br>
USC ID: 5483611649

## Project Packages

In [1]:
# TensorFlow and Keras 
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, ResNet101, EfficientNetB0, VGG16
from tensorflow.keras.layers import Activation, Input, Dense, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D, Rescaling, Resizing, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, RandomFlip, RandomContrast
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint

# Sklearn 
from sklearn.metrics import classification_report, roc_auc_score

# Utility 
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import logging
tf.get_logger().setLevel(logging.ERROR) # Remove INFO messages


## Data Exploration and Pre-processing

First we create all the dataset with a 20% cross validation split and one-hot encoding. We also stablish basic parameters like Image size and batch. Seed 42 is used for consistency.

In [2]:
# Data
train_dir =  '../Data/seg_train'
test_dir =  '../Data/seg_test'

# Parameters
IMG_SIZE = (224, 224) 
BATCH_SIZE = 32 # Good balance

# Training dataset
train = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,  # Resize 
    batch_size=BATCH_SIZE,  # Batch 
    label_mode="categorical",  # One-hot encoding 
    validation_split=0.2,  # 20% validation
    subset="training",  # Subset
    seed=42  
)

# Validation dataset
val = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    validation_split=0.2,  
    subset="validation",
    seed=42
)

# Test dataset
test = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical" 
)

print("")
classes = train.class_names
num_classes = len(classes)
print(f"Class names: {classes}")
print(f"Number of classes: {num_classes}")

print("\nOne-hot encoding Test")
for images, labels in train.take(1):  
    print("Sample:", labels[0]) 
    break


NotFoundError: Could not find directory ../Data/seg_train

## Data Augmentation

We apply sevelar augmentation techniques sequentialy (with Keras):
    
    Random cropping
    Random translation
    Random Rotation
    Random Flipping
    Brightness
    Random Contrast 
    Random Zoom


In [ ]:
# Augmentation pipeline
pipeline = Sequential(
  [    
        # Croping gives considerably bad results
        #RandomCrop(height=200, width=200), # Crop and resize
        #Resizing(224, 224),    
        RandomFlip("horizontal"), # Flip H
        RandomRotation(factor=0.05), # Rotations 
        RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1)), # Zoom 
        RandomTranslation(height_factor=0.1, width_factor=0.1), # Translations
        RandomContrast(factor=0.1), # Contrast 
        tf.keras.layers.Lambda(lambda x: tf.image.random_brightness(x, max_delta=0.1)), # Brightness
    ], 
    name = "pipeline",
)

## Transfer Learning

We create a function campable of creating a "Frozen" model for the 4 targeted types, 256 units, ReLU activation, L2 Regularization, Batch normalization, softmax output, and 20% dropout.

In [ ]:
# Model creation

model_mapping = {
    "EfficientNetB0": EfficientNetB0,
    "VGG16": VGG16,
    "ResNet50": ResNet50,
    "ResNet101": ResNet101
}

def create_model(model_type, input_shape):
    inputs = Input(shape = input_shape)
    x = pipeline(inputs)
    basic_model = model_mapping[model_type](weights="imagenet", include_top=False, input_tensor = x)

    # Freeze 
    basic_model.trainable = False

    # Head
    x = GlobalAveragePooling2D(name = "avg_pool")(basic_model.output) 
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)    
    outputs = Dense(6, activation = "softmax", kernel_regularizer=l2(0.01))(x)
    return Model(inputs, outputs)


##  Model Training

We train the models using multinomial cross entropy as well as ADAM optimizer. Epoch = 100 as default. Use of learning rate scheduler. Early stopping at 10 instances without improvement,

In [ ]:
def train_model(model, train_data, val_data, test_data, epochs, name, lr=0.001):
    
    # Learning rate scheduler
    lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=3,
        min_lr=1e-6
    )
        
    # Set model
    model.compile(
        optimizer=Adam(lr), # ADAM
        loss="categorical_crossentropy", # Multinomial cross entropy
        metrics=["accuracy"]
    )

    # Model checkpoint callback
    checkpoint_filepath = f'../ModelCheckpoints/{name}.h5'
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,  
        monitor='val_loss', 
        save_best_only=True,  
        save_weights_only=False
    )

    # Train model
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        callbacks=[lr_schedule, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)] # Early stopping
    )

    return history, model

### ResNet50 Train

In [ ]:
INPUT_SHAPE = (224, 224, 3)
histories = {}
models ={}
best ={}

print("Training ResNet50")
models["ResNet50"]  = create_model("ResNet50", INPUT_SHAPE)
histories["ResNet50"],best["ResNet50"]  = train_model(models["ResNet50"], train, val, test, 1, "ResNet50", 0.001)

### ResNet101 Train

In [ ]:
print("Training ResNet101")
models["ResNet101"]  = create_model("ResNet101", INPUT_SHAPE)
histories["ResNet101"], best["ResNet101"]= train_model(models["ResNet101"], train, val, test, 70, "ResNet101", 0.001)

### VGG16 Train

In [ ]:
print("Training VGG16")
models["VGG16"] = create_model("VGG16", INPUT_SHAPE)
histories["VGG16"], best["VGG16"] = train_model(models["VGG16"], train, val, test, 70, "VGG16", 0.001)

### EfficientNetB0 Train

In [ ]:
print("Training EfficientNetB0:")
models["EfficientNetB0"] = create_model("EfficientNetB0", INPUT_SHAPE)
histories["EfficientNetB0"], best["EfficientNetB0"] = train_model(models["EfficientNetB0"], train, val, test, 70, "EfficientNetB0", 0.001)

# Evaluation

Charts and scores for each model.

In [ ]:
def eval_model(model, train_data, val_data, test_data, history):
    def calculate_metrics(model, dataset, name):
        predictions = []
        true_labels = []

        for x, y in dataset:
            # Collect probabilities (predicted scores) and true labels
            probs = model.predict(x, verbose=0)  # Predicted probabilities
            predictions.extend(probs)  # Add probabilities
            true_labels.extend(y.numpy())  # Add one-hot encoded labels

        # Convert to NumPy arrays
        predictions = np.array(predictions)  # Shape: (num_samples, num_classes)
        true_labels = np.array(true_labels)  # Shape: (num_samples, num_classes)

        # Classification report using argmax
        pred_labels = np.argmax(predictions, axis=-1)
        true_labels_argmax = np.argmax(true_labels, axis=-1)
        report = classification_report(true_labels_argmax, pred_labels, target_names=classes, output_dict=True)
        print(f"{name} Metrics:\n", classification_report(true_labels_argmax, pred_labels, target_names=classes))

        # ROC AUC score (macro or weighted)
        roc_auc = roc_auc_score(true_labels, predictions, multi_class="ovr", average="macro")
        print(f"{name} ROC AUC Score: {roc_auc:.4f}")

        return report, roc_auc

    # Plot training vs validation loss
    plt.figure(figsize=(10, 6))
    plt.plot(history.history["loss"], label="Training Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title("Training vs Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    
    # Evaluate metrics for each dataset
    print("Evaluating Training Data...")
    train_report, train_auc = calculate_metrics(model, train_data, "Training")
    print("\nEvaluating Validation Data...")
    val_report, val_auc = calculate_metrics(model, val_data, "Validation")
    print("\nEvaluating Test Data...")
    test_report, test_auc = calculate_metrics(model, test_data, "Test")



    # Optionally return metrics if needed for further analysis
    return {
        "Train": {"report": train_report, "auc": train_auc},
        "Validation": {"report": val_report, "auc": val_auc},
        "Test": {"report": test_report, "auc": test_auc},
    }


In [ ]:
metrics_summary = []
for model_type in  ["ResNet50","ResNet101","VGG16","EfficientNetB0"]:
    print(f"Evaluation {model_type}:")
    results = eval_model(best[model_type], train, val, test, histories[model_type])
    
     # Extract metrics for each dataset
    for dataset_name, metrics in results.items():
        report = metrics["report"]["weighted avg"]
        auc = metrics["auc"]
        
        # Append metrics to summary list
        metrics_summary.append({
            "Model": model_type,
            "Dataset": dataset_name,
            "Precision": report["precision"],
            "Recall": report["recall"],
            "F1 Score": report["f1-score"],
            "AUC": auc,
        })
    print("\n\n\n")
metrics_df = pd.DataFrame(metrics_summary)

In [ ]:
# Display the DataFrame
metrics_df

## Observations

AA